In [35]:
import respy as rp
import pandas as pd
import numpy as np

In [64]:
params, options, data = rp.get_example_model("kw_94_two")

In [110]:
def add_generic_occupations(params, num_occupations):

    params_occ = params.copy()
    choices = params.index.unique("category")

    occupations = list()
    for choice in choices:
        if "wage_" not in choice:
            continue   
        occupations.append(choice.split("_")[1])

    # Now we just pick the first one.
    occ_generic = params.loc[f"wage_{occupations[0]}"]
    occ_generic = pd.concat([occ_generic], keys=["wage_a"], names=["category"])

        # Now we remove all experience terms.
    for index in occ_generic.index.get_level_values(0):

        if "exp_" not in index:
            continue

        if "edu" in index:
            continue

        occ_generic.drop(index, level="name", inplace=True)    


    # Now we create a new experience variables
    for letter in letters:
        for ext_ in ["", "_square"]:
            name = f"exp_{letter}" + ext_
            info = {
                "category": ["wage_a"],
                "name": [name],
                "value": [0.0],
                "comment": ["comment"],
            }

            info = pd.DataFrame.from_dict(info).set_index(["category", "name"])
            occ_generic = occ_generic.append(info)
    
    # Now we just put them all together.
    for letter in letters:
        occ_letter = occ_generic.loc["wage_a", :].copy()
        occ_letter = pd.concat([occ_letter], keys=[f"wage_{letter}"], names=["category"])##
        params_occ = params_occ.append(occ_letter)

    return params_occ

In [112]:
num_occupations = 3
params_occ = get_generic_occupations(params, num_occupations)

MultiIndex([(              'delta',                   'delta'),
            (             'wage_a',                'constant'),
            (             'wage_a',                 'exp_edu'),
            (             'wage_a',                   'exp_a'),
            (             'wage_a',            'exp_a_square'),
            (             'wage_a',                   'exp_b'),
            (             'wage_a',            'exp_b_square'),
            (             'wage_b',                'constant'),
            (             'wage_b',                 'exp_edu'),
            (             'wage_b',                   'exp_b'),
            (             'wage_b',            'exp_b_square'),
            (             'wage_b',                   'exp_a'),
            (             'wage_b',            'exp_a_square'),
            (         'nonpec_edu',                'constant'),
            (         'nonpec_edu', 'at_least_twelve_exp_edu'),
            (         'nonpec_edu',     

In [80]:
occ_generic

value                                       comment
category name                                                              
wage_a   constant      9.2100                           log of rental price
         exp_edu       0.0400     return to an additional year of schooling
         exp_a         0.0330              return to same sector experience
         exp_a_square -0.0005   return to same sector, quadratic experience
         exp_b         0.0000             return to other sector experience
         exp_b_square  0.0000  return to other sector, quadratic experience
         exp_a         0.0000                                       comment
         exp_a_square  0.0000                                       comment
         exp_b         0.0000                                       comment
         exp_b_square  0.0000                                       comment
         exp_c         0.0000                                       comment
         exp_c_square  0.0000                                       comment